In [1]:
from requests_html import HTMLSession
import json


In [2]:
with open("urls1.txt", "r") as f:
    string = f.read()
    # stringList = string.split("https")
    # stringList.pop(0)
    # stringFnl = list(map(lambda x: "https"+x, stringList))


In [3]:
stringFnl = string.split('\n')
stringFnl.pop()
len(stringFnl)


20072

In [4]:
stringFnl.index('https://hk.centanet.com/findproperty/zh-cn/transaction-detail/-%E7%A4%BC%E9%A1%BF%E5%B1%B1_V.R202201S0198')

937

In [5]:
import pandas as pd
df = pd.read_csv('data1.csv')

In [6]:
df.dropna()

,Unnamed: 0,rooms,age,direction,priceNum,priceLbl,area,location,gfa,cfa
0,0,2房,35 年,东南,618,万,沙田,伟华中心 4座 中层 E室,358,483
2,2,3房,28 年,东北,720,万,马鞍山,新港城 3期 R座 低层 4室,503,674
3,3,1房,39 年,东北,510,万,沙田,好运中心 枫林阁 (C1座) 低层 H室,326,426
5,5,2房,39 年,西北,613.8,万,沙田,好运中心 榆林阁 (A2座) 8楼 H室,427,555
6,6,2房,36 年,东北,565,万,荃湾市中心,荃锦中心 1座 12楼 A室,369,511
...,...,...,...,...,...,...,...,...,...,...
6220,6220,2房,24 年,西北,747.7,万,坑口,东港城 8座 31楼 C室,463,588
6226,6226,3房,11 年,东南,"1,275",万,元朗站,YOHO TOWN 2期 YOHO MIDTOWN 8座 9楼 D室,893,"1,148"
6229,6229,2房,11 年,东北,818,万,元朗站,YOHO TOWN 2期 YOHO MIDTOWN 9座 27楼 A室,525,676
6233,6233,2房,18 年,东南,668,万,元朗站,YOHO TOWN 1期 9座 20楼 E室,422,544


In [7]:
stringFnl[0]



'https://hk.centanet.com/findproperty/zh-cn/transaction-detail/-%E4%BC%9F%E5%8D%8E%E4%B8%AD%E5%BF%83_FTY202203S0762'

In [8]:
stringFnl[1]

'https://hk.centanet.com/findproperty/zh-cn/transaction-detail/-%E9%92%9C%E5%8F%91%E5%A4%A7%E5%8E%A6_YK1202204S0187'

In [9]:
idx = 6234

In [10]:
def collect_info(url):
    r = session.get(url, headers = header)
    info = dict()
    info["rooms"] = None
    info["age"] = None
    info["direction"] = None
    info["priceNum"] = None
    info["priceLbl"] = None
    info["area"] = None
    info["location"] = None
    info["gfa"] = None
    info["cfa"] = None
    
    #price
    try:
        priceNum = r.html.xpath("//span[@class='price-lbl']/text()")[0].strip()
        priceLbl = r.html.xpath(
            "//span[@class='price-lbl']/label/text()")[0].strip()
        info["priceNum"] = priceNum
        info["priceLbl"] = priceLbl
    except IndexError:
        pass

    #location and area
    try:
        area = r.html.xpath(
            "//span[@class='el-breadcrumb__item'][5]/span/a/text()")[0].strip()
        info["area"] = area
    except IndexError:
        pass
    
    try:
        location = r.html.xpath(
            "//h1[@class='info-title f-middle']/p/text()")[0].strip()
        info["location"] = location
    except IndexError:
        pass
    
    #gfa cfa
    try:
        gfa = r.html.xpath("//div[@class='fl']/p/span[2]/text()")[0].strip()
        info["gfa"] = gfa
    
    except IndexError:
        pass
    try:
        cfa = r.html.xpath("//div[@class='fr']/p/text()")[0].strip()
        info["cfa"] = cfa
    except IndexError:
        pass
    #rooms age direction
    
    for divID in range(len(r.html.xpath("// div[@class='info-num flex-between']/div"))):
        div = divID+1
        cat = r.html.xpath(
            "// div[@class='info-num flex-between']/div[{}]/p[1]/text()".format(div))[0].strip()
        content = r.html.xpath(
            "// div[@class='info-num flex-between']/div[{}]/p[2]/text()".format(div))[0].strip()
        if cat == "楼龄":
            info["age"] = content
        elif cat == "座向":
            info["direction"] = content
        elif cat == "间隔":
            info["rooms"] = content

    return info


In [11]:
session = HTMLSession()

header = {

    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
}


In [12]:
from tqdm import tqdm
import pandas as pd

In [13]:
import time
data_lst = []
fail_lst = []
j = 7001
pbar = tqdm(total = len(stringFnl))
while j < len(stringFnl):
    if (j%100==0):
        df = pd.DataFrame(data_lst)
        df.to_csv("data.csv")
    if (j%1000==0):
        df = pd.DataFrame(data_lst)
        df.to_csv("data_"+str(j)+".csv")
    try:
        i = stringFnl[j]
        d = collect_info(i)
        time.sleep(5)
        if d['location'] == None:
            count = 0
            print("retrying:")
            print(i)
            # while count <= 5 and d['location'] == None:
            #     count += 1
            #     time.sleep(5+count)
            d = collect_info(i)
            if d['location'] == None:
                print('fail')
                fail_lst.append(i)
                j+=1
                pbar.update(1)
                continue
            else:
                print('success')
        data_lst.append(d)
        count = 0
        j+=1
        pbar.update(1)
    except:
        time.sleep(30)
pbar.close()


 15%|█▌        | 3080/20072 [4:40:30<25:23:51,  5.38s/it]

retrying:
https://hk.centanet.com/findproperty/zh-cn/transaction-detail/-_22042900500088


 15%|█▌        | 3081/20072 [4:41:35<110:06:48, 23.33s/it]

fail


 16%|█▌        | 3168/20072 [4:51:26<24:46:23,  5.28s/it] 

...

 20%|█▉        | 3966/20072 [6:02:24<23:53:23,  5.34s/it]

success


 20%|██        | 4089/20072 [6:13:20<23:21:37,  5.26s/it]

retrying:
https://hk.centanet.com/findproperty/zh-cn/transaction-detail/-%E6%96%B0%E5%AE%9D%E5%9F%8E_KUK3202201S0143
fail


 38%|███▊      | 7554/20072 [11:44:13<18:45:20,  5.39s/it]  

In [ ]:
len(data_lst)

234

In [2]:
import pandas as pd  
df = pd.read_csv('data.csv')

In [3]:
df.dropna()

,Unnamed: 0,rooms,age,direction,priceNum,priceLbl,area,location,gfa,cfa
6,6,3房,38 年,西北,835,万,绿杨,绿杨新邨 C座 30楼 8室,582,671
12,12,2房,35 年,西北,558,万,荃湾市中心,荃丰中心 19楼 B室,336,447
15,15,2房,20 年,西北,623,万,元朗站,采叶庭 11座 2楼 F室,509,673
18,18,2房,8 年,西南,568,万,元朗东南,尚悦 5座 10楼 B室,403,547
20,20,2房,13 年,东,831.6,万,太子,百汇轩 9楼 B室,468,670
...,...,...,...,...,...,...,...,...,...,...
7450,7450,3房,21 年,西北,"1,000",万,马鞍山,迎涛湾 1座 (A座) 25楼 B室,736,959
7452,7452,2房,16 年,北,"1,000",万,坚尼地城,域多利道60号 27楼 B室,466,633
7459,7459,3房,32 年,东南,658,万,荃湾市中心,豪辉花园 2座 8楼 D室,437,699
7464,7464,3房,38 年,东南,850,万,绿杨,绿杨新邨 F座 10楼 4室,582,667


In [ ]:
cnt = 0
with open('data.json',mode='w', encoding='utf-8') as f:
    for i in stringFnl:
        cnt+=1
        #print(cnt,"working with:",i)
        print("*",end="")
        info = collect_info(i)
        json.dump(info,f)
        #print("done")

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [ ]:
collect_info("https://hk.centanet.com/findproperty/zh-cn/transaction-detail/-%E6%B5%B7%E8%81%94%E5%B9%BF%E5%9C%BA_22030401730038")


{'rooms': None,
 'age': '27 年',
 'direction': None,
 'priceNum': '495',
 'priceLbl': '万',
 'area': '粉岭',
 'location': '海联广场 2座 5楼 G室',
 'gfa': '442',
 'cfa': '584'}

In [ ]:
len("***************")


15

In [ ]:
with open("data.json","r") as f:
    s = f.readlines()[0]

In [ ]:
len(s)

1433015

In [ ]:
lst = s.split("{")

In [ ]:
lst[1]

'"rooms": "2\\u623f", "age": "17 \\u5e74", "direction": "\\u4e1c\\u5317", "priceNum": "898", "priceLbl": "\\u4e07", "area": "\\u5751\\u53e3", "location": "\\u851a\\u84dd\\u6e7e\\u7554 5\\u5ea7 \\u4e2d\\u5c42 G\\u5ba4", "gfa": "490", "cfa": "657"}'

In [ ]:
lst = list(map(lambda x:"{"+x, lst))

In [ ]:
lst1 = list(map(lambda x :json.loads(x), lst))

In [ ]:
with open("data1.json","w") as f:
    json.dump(lst1,f)


In [ ]:
f.close()

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(lst1)

In [ ]:
df.iloc[:10]

,rooms,age,direction,priceNum,priceLbl,area,location,gfa,cfa
0,2房,21 年,北,978.8,万,坚尼地城,加惠台 1座 低层 C室,464,589
1,2房,17 年,东北,898,万,坑口,蔚蓝湾畔 5座 中层 G室,490,657
2,None,49 年,None,494,万,湾仔,永德大厦 B座 4楼 B8室,373,500
3,None,33 年,东,455,万,元朗市中心,荣丰大厦 6楼 B室,423,616
4,None,27 年,西,"1,000",万,马鞍山,雅典居 7座 17楼 D室,727,901
5,2房,8 年,西北,618,万,元朗东南,尚悦 1座 19楼 D室,425,576
6,None,27 年,西,"1,038",万,马鞍山,雅典居 9座 22楼 D室,727,901
7,None,31 年,None,662,万,粉岭,粉岭中心 1期 G座 中层 3室,535,694
8,None,57 年,南,"1,360",万,何文田,麒麟阁 低层 E室(244号),"1,075","1,293"
9,2房,22 年,东北,718,万,坑口,新宝城 1座 中层 G室,377,527
